In [ ]:
# Parameters
inpath = ""
outpath = ""


# XAS Notebook

The XAS Notebook autoprocessor will run automatically on every scan performed that contains the [NXxas](https://manual.nexusformat.org/classes/applications/NXxas.html#nxxas) sub-entry.

A simple analysis and plot of the data is provided.

Note: [mmg_toolbox](https://github.com/DanPorter/mmg_toolbox) is in active development and likely to change in the future, therefore please don't rely to heavily on the functions at this point. You can see how everything works at the repository. If you have any comments, please do get in touch!

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt

import hdfmap  # available via pip install hdfmap
print(hdfmap.module_info())


In [ ]:
# Load data from NeXus file using HdfMap
# Works across multiple NeXus file types - i06, i10 etc

nxs_map = hdfmap.create_nexus_map(inpath)

with nxs_map.load_hdf() as nxs:
    def rd(expr, default=''):
        return nxs_map.format_hdf(nxs, expr, default=default)

    # currently accounts for i06-1 and i10-1 metadata
    metadata = {
        "scan": rd('{filename}'),
        "cmd": rd('{(cmd|user_command|scan_command)}'),
        "title": rd('{title}', os.path.basename(inpath)),
        "endstation": rd('{end_station}', 'unknown'),
        "sample": rd('{sample_name}', ''),
        "energy": rd('{mean((fastEnergy|pgm_energy|energye|energyh)):.2f} eV'),
        "pol": rd('{polarisation?("lh")}'),
        "height": rd('{(em_y|hfm_y):.2f}', 0),
        "pitch": rd('{(em_pitch|hfm_pitch):.2f}', 0),
        "temperature": rd('{(T_sample|sample_temperature|lakeshore336_cryostat|lakeshore336_sample|itc3_device_sensor_temp?(300)):.2f} K'),
        "field": rd('{(field_z|sample_field|magnet_field|ips_demand_field?(0)):.2f} T'),
    }

    print('Scan data paths:')
    print('energy: ', nxs_map.eval(nxs, '_(fastEnergy|pgm_energy|energye|energyh)'))
    print('monitor: ', nxs_map.eval(nxs, '_(C2|ca62sr|mcs16_data|mcse16_data|mcsh16_data)'))
    print('tey: ', nxs_map.eval(nxs, '_(C1|ca61sr|mcs17_data|mcse17_data|mcsh17_data)'))
    print('tfy: ', nxs_map.eval(nxs, '_(C3|ca63sr|mcs18_data|mcse18_data|mcsh18_data|mcsd18_data)'))

    energy = nxs_map.eval(nxs, '(fastEnergy|pgm_energy|energye|energyh)')
    monitor = nxs_map.eval(nxs, '(C2|ca62sr|mcs16_data|mcse16_data|mcsh16_data)', default=1.0)
    tey = nxs_map.eval(nxs, '(C1|ca61sr|mcs17_data|mcse17_data|mcsh17_data)', default=np.ones(nxs_map.scannables_shape())) / monitor
    tfy = nxs_map.eval(nxs, '(C3|ca63sr|mcs18_data|mcse18_data|mcsh18_data|mcsd18_data)', default=np.ones(nxs_map.scannables_shape())) / monitor

print('\nMetadata:')
print('\n'.join(f"{n:12}: {d}" for n, d in metadata.items()))

title = "{endstation} {sample} {scan}\nE = {energy}, pol = {pol}, T = {temperature}, B = {field}".format(**metadata)
print('\ntitle: ', title)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle(title)

ax[0].plot(energy, tey, '-', label=f"{metadata['pol']} TEY")
ax[1].plot(energy, tfy, '-', label=f"{metadata['pol']} TFY")

ax[0].set_xlabel('Energy [eV]')
ax[0].set_ylabel('signal / monitor')
ax[0].legend()

ax[1].set_xlabel('Energy [eV]')
ax[1].set_ylabel('signal / monitor')
ax[1].legend()

# Analyse absorption edge

1. normalise by pre-edge
2. fit step-edge background
3. remove background and normalise to step-edge
4. save NeXus file

In [ ]:
# Get absorption edge from energy
from mmg_toolbox.spectra_analysis import energy_range_edge_label, xray_edges_in_range

print(f"Absorption edges between: {energy.min()}, {energy.max()} eV")
print('\n'.join(f"{en} eV : {lab}" for en, lab in xray_edges_in_range(energy.min(), energy.max(), search_edges=None)))

edge_label = energy_range_edge_label(energy.min(), energy.max())
print('Automatically determined absorption edge:')
print(f"\nEdge label: {' '.join(edge_label)}")

In [ ]:
# Load scan as Spectra object
from mmg_toolbox.nxxas_loader import load_from_nxs_using_hdfmap
scan = load_from_nxs_using_hdfmap(inpath)
print(scan)

In [ ]:
# Plot raw spectra
fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=80)
fig.suptitle('raw scan files')
for n, (mode, spectra) in enumerate(scan.spectra.items()):
    spectra.plot(ax=axes[n], label=scan.name)
    axes[n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()

In [ ]:
# 1. Normalise by pre-edge
scan.divide_by_preedge()

# plot scan normalised scan files
fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=80)
fig.suptitle('Normalise by pre-edge')
for n, (mode, spectra) in enumerate(scan.spectra.items()):
    spectra.plot(ax=axes[n], label=scan.name)
    axes[n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()


In [ ]:
# 2. Fit and subtract background
scan.auto_edge_background(peak_width_ev=10.)

# Plot background subtracted scans
fig, axes = plt.subplots(2, 2, figsize=(12, 10), dpi=80)
fig.suptitle(scan.name)
for n, (mode, spectra) in enumerate(scan.spectra.items()):
    spectra.plot_parents(ax=axes[0, n])
    spectra.plot_bkg(ax=axes[0, n])
    axes[0, n].set_ylabel(mode)

    spectra.plot(ax=axes[1, n], label=scan.name)
    axes[1, n].set_ylabel(mode)

for ax in axes.flat:
    ax.set_xlabel('E [eV]')
    ax.legend()

In [ ]:
# 4. Save Nexus file
#scan.write_nexus(outpath)

# Search for previous NXxas scans
If the previous scan was also a spectra scan, plot this and take the difference

In [ ]:
from mmg_toolbox.nxxas_loader import find_similar_measurements

# check previous ~10 scans and build a list of energy scans at the same edge, temperature and field
# see help(find_similar_measurements) for more info
scans = find_similar_measurements(inpath)
pols = {s.metadata.pol for s in scans}

print(f"Unique polarisations: {pols}")

if len(pols) > 1:
    print('\n\n'.join(str(s) for s in scans))
else:
    print('Not enough polarisations')

In [ ]:
# Repeat steps 1-3 for each scan
if len(pols) > 1:
    for scan in scans:
        scan.divide_by_preedge()
        print(scan)

    # plot scan normalised scan files
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=80)
    fig.suptitle('Normalise by pre-edge')
    for scan in scans:
        for n, (mode, spectra) in enumerate(scan.spectra.items()):
            spectra.plot(ax=axes[n], label=scan.name)
            axes[n].set_ylabel(mode)

    for ax in axes.flat:
        ax.set_xlabel('E [eV]')
        ax.legend()
else:
    print('Not enough polarisations')


In [ ]:
# Fit and subtract background
if len(pols) > 1:
    for scan in scans:
        scan.auto_edge_background(peak_width_ev=10.)

    # Plot background subtracted scans
    for scan in scans:
        fig, axes = plt.subplots(2, 2, figsize=(12, 10), dpi=80)
        fig.suptitle(scan.name)
        for n, (mode, spectra) in enumerate(scan.spectra.items()):
            spectra.plot_parents(ax=axes[0, n])
            spectra.plot_bkg(ax=axes[0, n])
            axes[0, n].set_ylabel(mode)

            spectra.plot(ax=axes[1, n], label=scan.name)
            axes[1, n].set_ylabel(mode)

        for ax in axes.flat:
            ax.set_xlabel('E [eV]')
            ax.legend()


In [ ]:
if len(pols) > 1:
    from mmg_toolbox.spectra_container import average_polarised_scans

    # Average polarised scans
    for scan in scans:
        print(f"{scan.name}: {scan.metadata.pol}")
    pol1, pol2 = average_polarised_scans(*scans)
    print(pol1)
    print(pol2)

    # Plot averaged scans
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=80)
    fig.suptitle('Averaged polarised scans')
    for scan in [pol1, pol2]:
        for n, (mode, spectra) in enumerate(scan.spectra.items()):
            spectra.plot(ax=axes[n], label=scan.name)
            axes[n].set_ylabel(mode)

    for ax in axes.flat:
        ax.set_xlabel('E [eV]')
        ax.legend()


# Calculate XMCD/XMLD

In [ ]:
if len(pols) > 1:
    # Calculate XMCD
    xmcd = pol1 - pol2
    print(xmcd)

    for name, spectra in xmcd.spectra.items():
        print(spectra)
        print(spectra.process)
        print(spectra.sum_rules_report(1))


In [ ]:
if len(pols) > 1:
    fig, axes = plt.subplots(1, 2, figsize=(12, 6), dpi=80)
    fig.suptitle(xmcd.name.upper())
    for n, (mode, spectra) in enumerate(xmcd.spectra.items()):
        spectra.plot(ax=axes[n])
        axes[n].set_ylabel(mode)

    for ax in axes.flat:
        ax.set_xlabel('E [eV]')
        ax.legend()

In [ ]:
if len(pols) > 1:
    # Save xmcd file
    xmcd_filename = f"{scans[0].metadata.scan_no}-{scans[-1].metadata.scan_no}_{xmcd.name}.nxs"
    output_folder = os.path.dirname(outpath)
    #xmcd.write_nexus(os.path.join(output_folder, xmcd_filename))